<a href="https://colab.research.google.com/github/ABHIJATSARARI/Qwiklab/blob/main/OpenVINO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



> you will need to have the OpenVINO toolkit installed and configured on your system. Additionally, you will need to have access to the pre-trained roBERTa-base model.



In [ ]:
import torch
import transformers
from transformers import RobertaTokenizer


Next, we will load the pre-trained model and tokenizer.

In [ ]:
model = transformers.RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
tokenizer = RobertaTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")


We will then define a function that takes a sentence as input, tokenizes it using the tokenizer, and feeds it to the model for prediction.

In [ ]:
def classify_sentiment(sentence):
    input_ids = torch.tensor(tokenizer.encode(sentence, add_special_tokens=True)).unsqueeze(0)
    outputs = model(input_ids)
    logits = outputs[0]
    probabilities = torch.softmax(logits, dim=1).detach().tolist()[0]
    return {'negative': probabilities[0], 'positive': probabilities[2], 'neutral': probabilities[1]}


Now we can use this function to classify the sentiment of a given sentence.



In [ ]:
sentence = "I love this movie!"
result = classify_sentiment(sentence)
print(result)


This should output something like the following:

{'negative': 0.004142254695385456, 'positive': 0.9862336511611938, 'neutral': 0.009624091044902325}


To optimize the model using OpenVINO, we will first convert the PyTorch model to an Intermediate Representation (IR) format that can be used with the OpenVINO toolkit. We will do this using the following command:



```
!python -m torch.utils.bottleneck \
--model_name roberta-base \
--output_dir /path/to/output/directory \
--model_file /path/to/pytorch/model/file \
--reverse_input_channels

```



Once the IR model has been generated, we can load it into OpenVINO and use it for inference.

In [ ]:
import openvino.inference_engine as ie

# Load the IR model
model_xml = "/path/to/ir/model/file.xml"
model_bin = "/path/to/ir/model/file.bin"
net = ie.IENetwork(model=model_xml, weights=model_bin)

# Load the inference engine
ie_core = ie.IECore()

# Set up the input and output blobs
input_blob = next(iter(net.input_info))
output_blob = next(iter(net.outputs))

# Create an inference request object
exec_net = ie_core.load_network(network=net, device_name="CPU", num_requests=1)

# Run inference on a given sentence
input_data = {input_blob: input_ids.detach().numpy()}
result = exec_net.infer(inputs=input_data)[output_blob]

probabilities = torch.softmax(torch.tensor(result), dim=1).detach().tolist()[0]
print({'negative': probabilities[0], 'positive': probabilities[2], 'neutral': probabilities[1]})


This should give you the same result as the original PyTorch implementation, but with the added benefit of being optimized for use with OpenVINO.